In [122]:
from glob import glob
import numpy as np
import librosa
from tqdm import tqdm
import os

In [123]:
audio_wav_path_list: list = glob(pathname="..\\data\\audio\\*\\*.wav", recursive=True)
print(f"type(audio_wav_path_list): {type(audio_wav_path_list)}")
print(f"len(audio_wav_path_list): {len(audio_wav_path_list)}")
# wav_path_list

type(audio_wav_path_list): <class 'list'>
len(audio_wav_path_list): 42142


In [124]:
record_wav_path_list: list = glob(pathname="..\\data\\record\\*.wav", recursive=True)
print(f"type(record_wav_path_list): {type(record_wav_path_list)}")
print(f"len(record_wav_path_list): {len(record_wav_path_list)}")
# record_wav_path_list

type(record_wav_path_list): <class 'list'>
len(record_wav_path_list): 110


In [125]:
edu_wav_path_list = glob(pathname="..\\data\\samplePinyinEdu\\*\\*.wav", recursive=True)
print(f"type(edu_wav_path_list): {type(edu_wav_path_list)}")
print(f"len(edu_wav_path_list): {len(edu_wav_path_list)}")
# edu_wav_path_list

type(edu_wav_path_list): <class 'list'>
len(edu_wav_path_list): 2933


In [126]:
edu_male_wav_path_list: list = glob(pathname="..\\data\\samplePinyinEdu\\Male\\*.wav")
edu_pinyin_list: list = [wav_path[wav_path.find("_") + 1: wav_path.find(".wav")] for wav_path in edu_male_wav_path_list]

print(f"type(edu_pinyin_list): {type(edu_pinyin_list)}")
print(f"len(edu_pinyin_list): {len(edu_pinyin_list)}")
# edu_pinyin_list

type(edu_pinyin_list): <class 'list'>
len(edu_pinyin_list): 1467


In [127]:
audio_wav_path_list_selected = list()

for wav_path in audio_wav_path_list:
    pinyin = wav_path[wav_path.find("_") + 1: wav_path.find(".wav")]
    modified_pinyin: str
    if not pinyin[-1].isdigit():
        modified_pinyin = pinyin + "5"
    elif pinyin[-1] == "1":
        modified_pinyin = pinyin[:-1]
    else:
        modified_pinyin = pinyin
    
    if modified_pinyin in edu_pinyin_list:
        audio_wav_path_list_selected.append(wav_path)

print(f"type(audio_wav_path_list_selected): {type(audio_wav_path_list_selected)}")
print(f"len(audio_wav_path_list_selected): {len(audio_wav_path_list_selected)}")
# audio_wav_path_list_selected

type(audio_wav_path_list_selected): <class 'list'>
len(audio_wav_path_list_selected): 42124


In [128]:
wav_path_list = list()
for path_list in [audio_wav_path_list_selected, record_wav_path_list, edu_wav_path_list]:
    for wav_path in path_list:
        wav_path_list.append(wav_path)

print(f"type(wav_path_list): {type(wav_path_list)}")
print(f"len(wav_path_list): {len(wav_path_list)}")
# wav_path_list

type(wav_path_list): <class 'list'>
len(wav_path_list): 45167


In [135]:
edu_mfcc_path_list = list()
i = 0
for wav_path in audio_wav_path_list_selected:
    pinyin = wav_path[wav_path.find("_") + 1: wav_path.find(".wav")]
    modified_pinyin: str
    if not pinyin[-1].isdigit():
        modified_pinyin = pinyin + "5"
    elif pinyin[-1] == "1":
        modified_pinyin = pinyin[:-1]
    else:
        modified_pinyin = pinyin
    
    path = wav_path
    path = path.replace(f"_{pinyin}.wav", f"_{modified_pinyin}_{i}.npy")
    i += 1
    path = path.replace(
        path[
            path.find("\\data\\audio\\") + len("\\data\\"):
            path.find("\\", (path.find("\\data\\audio\\") + len("\\data\\audio\\")))
        ],
        "mfccEduVer"
    )
    edu_mfcc_path_list.append(path)
    
for wav_path in record_wav_path_list:
    path = wav_path
    path = path.replace(f".wav", f"_{i}.npy")
    i += 1
    path = path.replace(
        path[
            path.find("record\\"):
            path.find("\\", (path.find("record\\")))
        ],
        "mfccEduVer"
    )
    edu_mfcc_path_list.append(path)
    
for wav_path in edu_wav_path_list:
    path = wav_path
    path = path.replace(f".wav", f"_{i}.npy")
    i += 1
    if "samplePinyinEdu\\Female\\" in wav_path:
        path = path.replace(
            path[
                path.find("samplePinyinEdu\\"):
                path.find("\\", (path.find("Female\\")))
            ],
            "mfccEduVer"
        )
    elif "samplePinyinEdu\\Male\\" in path:
        path = path.replace(
            path[
                path.find("samplePinyinEdu\\"):
                path.find("\\", (path.find("Male\\")))
            ],
            "mfccEduVer"
        )
    edu_mfcc_path_list.append(path)

print(f"type(edu_mfcc_path_list): {type(edu_mfcc_path_list)}")
print(f"len(edu_mfcc_path_list): {len(edu_mfcc_path_list)}")
# edu_mfcc_path_list

type(edu_mfcc_path_list): <class 'list'>
len(edu_mfcc_path_list): 45167


In [130]:
def get_mfcc(file_path, n_mfcc=13, max_pad_len=44, n_fft=2048, sr=22050, fmax=None, n_mels=128):
    # 讀取音檔，轉為單聲道
    audio, sample_rate = librosa.load(file_path, mono=True, sr=sr)

    # 確保 n_fft 小於或等於輸入信號的長度
    n_fft = min(n_fft, len(audio))

    # 計算梅爾頻率倒譜係數（MFCC）
    mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=n_mfcc, n_fft=n_fft, n_mels=n_mels, fmax=fmax)

    # 計算填充或截斷的長度
    pad_width = max_pad_len - mfccs.shape[1]
    if pad_width < 0:
        # 截斷
        mfccs = mfccs[:, :max_pad_len]
    else:
        # 填充
        mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    return mfccs

In [131]:
mfcc_matrix_list = list()

for wav_path in tqdm(wav_path_list):
    mfcc_matrix = get_mfcc(file_path=wav_path)
    mfcc_matrix_list.append(mfcc_matrix)

print(f"type(mfcc_matrix_list): {type(mfcc_matrix_list)}")
print(f"len(mfcc_matrix_list): {len(mfcc_matrix_list)}")
# mfcc_matrix_list

100%|██████████| 45167/45167 [01:42<00:00, 442.69it/s]

type(mfcc_matrix_list): <class 'list'>
len(mfcc_matrix_list): 45167


In [132]:
mfcc_folder = "..\\data\\mfccEduVer\\"
if not os.path.exists(mfcc_folder):
    os.makedirs(mfcc_folder)

for i in tqdm(range(len(edu_mfcc_path_list))):
    npy_path = edu_mfcc_path_list[i]
    mfcc_matrix = mfcc_matrix_list[i]
    np.save(file=npy_path, arr=mfcc_matrix)

100%|██████████| 45167/45167 [00:06<00:00, 7381.11it/s]
